# Intuitive Explanation of Beta Binomial Model
The beta binomial model is frequently talked about within the DSAA team and at Clover (see [Quip: Evaluating Percentage-Type Measurements (Beta Binomial)](https://cloverhealth.quip.com/f1N6ALVm6xXN)). This notebook offers an intuitive explanation for what it is and why it's useful.

In short, we generally use the beta binomial model in a bayesian framework to estimate the posterior distribution of some percentage type measurement given data that we've collected.

## The Mandatory Section on Bayes Theorem

Bayesian frameworks take advantage of Bayes' Theorem to do two important things:

* Generate the probability of a given Model (or set of model parameters) given the data collected.
* Update that probability as more data comes in.

Bayes Theorem of course is:

$$P(H|D) = \frac{P(D|H)P(H)}{P(D)}$$

1. $P(H|D)$: Posterior -- The probability of our hypothesis being true given our data collected
2. $P(D|H)$: Likelihood -- Probability of collecting our data given our hypothesis is true
3. $P(H)$: Prior -- The probability of the hypothesis being true prior to collecting the data
4. $P(D)$: Marginal -- What is the probability of collecting this data under all possible hypotheses?

## Okay, but what is the beta distribution?

The short version is that the Beta distribution can be understood as representing a distribution of probabilities. Let's go through a basic example.

Suppose we have a coin, but we have no idea if it's fair or not. In fact, let's say that the probability of flipping heads for the coin is completely unknown. Now, imagine that we actually want to estimate what that probability will be. A naive measure could be just the currently observed proportion of heads flipped, but this can be a bad measure for several reasons:

1. **Suffers from small samples**: if we flip a coin once and flip a heads, the proportion of heads flipped is briefly 1.000, while if we flip tails, the proportion of heads flipped is 0.000.
2. **Impossible to incorporate a prior**: If we know that the coin comes from a certain manufacturer such that the coin is very likely to be fair (or unfair) we should be able to incoporate that information somehow.
3. **Returns a point estimate not a distribution**: Just taking the current proportion of heads gives us only the point estimate. There's no structured way of estimating the variation or distribution of the estimated probability of flipping heads.



Our coin flipping problem is best represented as a binomial distribution, i.e. some series of successes (heads) and observations (flips). It also turns out that a really good way to represent the **prior** for the probability of heads is the *beta* distribution. The domain of the Beta distribution is (0, 1), just like a probability, so we already know we're on the right track- but the appropriateness of the Beta for this task goes far beyond that. 

Suppose we have a completely uninformed prior, that is, we have no idea about probability of the coin flipping heads. In that case, it would be best to model the prior as a uniform distribution, which can be represented with a beta distribution with parameters $\alpha = 1$ and $\beta = 1$.

Let's actually plot the beta distribution with those values for alpha and beta).

In [ ]:
import sys
sys.path.append('../../../')

import matplotlib.pyplot as plt
%matplotlib inline

from clover.dsa.tools import beta_binomial as bb

In [ ]:
distribution_values = bb.beta_values(alpha=1, beta=1)

In [ ]:
plt.scatter(distribution_values[0], distribution_values[1])
plt.xlabel('Probability of Coin Flipping Heads')
plt.title('PDF of Prior distribution of Coin Flipping Heads')
plt.show()

Also note that the expected value of the distribution is equal to 0.5 (which we know to be the case of a uniform distribution between 0 and 1). Also notice that,

$$E(p) = 0.5 = \frac{1}{1 + 1} = \frac{\alpha}{\alpha + \beta}$$

This is indeed one of the very nice mathematical properties of the beta distribution, that the expected value of a beta distribution is equal to $\frac{\alpha}{\alpha + \beta}$!

Finally, note that the Beta distribution is representing a probability distribution of a "percentage-type" measurement, in this case the percentage of flips that result in heads, or equivalently, the probability that the coin will flip head.

But here's why the Beta distribution is so appropriate. Imagine we go through 20 coin flips with 12 heads. The coin's record is now 12 heads; 20 flips. We have to then update our probabilities - we want to shift this entire curve over just a bit to reflect our new information. While the math for proving this is a bit involved ([it's shown here](https://en.wikipedia.org/wiki/Conjugate_prior#Example)), the result is very simple. The **posterior** Beta distribution will be: $Beta(\alpha_0 + successes, \beta_0 + failures)$, where $\alpha_0$ and $\beta_0$ are the initial parameters for the beta distribution for the **prior**, in this case, 1 and 1 respectively.

Thus, in this case, $\alpha$ has increased by 12, while $\beta$ has increased by 8. That means our new distribution is $Beta(1 + 12, 1 + 8)$ or:

In [ ]:
posterior_distribution_values = bb.beta_values(alpha=1 + 12, beta=1 + 8)
plt.scatter(posterior_distribution_values[0], posterior_distribution_values[1])
plt.xlabel('Probability of Coin Flipping Heads')
plt.xlim(0, 1)
plt.title('PDF of Posterior distribution of Coin Flipping Heads')
plt.show()

Notice that the posterior PDF has now updated the prior with our new information to be centered close to 0.6. In fact, the exact expected value is now 13/(13 + 9) = 0.59.

However, what if many coin flips go by? In fact, let's say that we flip 200 times with 120 heads! We would then update our prior and our posterior distribution would be $Beta(1 + 120, 1 + 80)$ or:

In [ ]:
posterior_distribution_values = bb.beta_values(alpha=1 + 120, beta=1 + 80)
plt.scatter(posterior_distribution_values[0], posterior_distribution_values[1])
plt.xlabel('Probability of Coin Flipping Heads')
plt.xlim(0, 1)
plt.title('PDF of Posterior distribution of Coin Flipping Heads')
plt.show()

Notice that what has happend now is that the expected value is still around 0.6, but the distribution has tightened. Intuitively, this makes a lot of sense because we have more observations so now we are more confident.

Now, just for illustrative purposes, let's go to the extreme case where we have 10000 coin flips with 6000 heads. The posterior look like: $Beta(1 + 6000, 1 + 4000)$

In [ ]:
posterior_distribution_values = bb.beta_values(alpha=1 + 6000, beta=1 + 4000)
plt.scatter(posterior_distribution_values[0], posterior_distribution_values[1])
plt.xlabel('Probability of Coin Flipping Heads')
plt.xlim(0, 1)
plt.title('PDF of Posterior distribution of Coin Flipping Heads')
plt.show()

Notice that now with so many observations, the distribution is extremely tight around 0.6 with very small variance. Again, this makes intuitive sense because we have so many observations that we can be very confident that the probability of the coin flipping heads is the expected value, 0.6, and thus the coin is very very likely not fair.